## Perform a 1D inversion 

This notebook reproduces the 1D inversion (Figure 5) shown in 

Heagy, L., Kang, S., Cockett, R., and Oldenburg, D., 2018, _Open source software for simulations and inversions of airborne electromagnetic data_, AEM 2018 International Workshop on Airborne Electromagnetics 

In [1]:
import numpy as np
from scipy.constants import mu_0
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import LogNorm

from pymatsolver import Pardiso
from SimPEG import (EM, Mesh, Maps, SolverLU, DataMisfit, Regularization,
                    Optimization, InvProblem, Inversion, Directives, Utils)

%matplotlib inline

In [2]:
matplotlib.rcParams['font.size'] = 16

In [3]:
dobs = np.load("dobs.npy")
x = np.load('x.npy')
times = np.logspace(np.log10(5e-5), np.log10(2.5e-3), 21)
nSrc = x.size
DOBS = dobs.reshape((nSrc, 2, times.size))

## Define the forward simulation mesh

In [4]:
def get_1d_cyl_mesh():
    cs, ncx, ncz, npad = 50., 10, 10, 10
    pad_rate = 1.3
    hx = [(cs, ncx),  (cs, npad, pad_rate)]
    hz = [(cs, npad, -pad_rate), (cs, ncz), (cs, npad, pad_rate)]
    mesh = Mesh.CylMesh([hx, 1, hz], '00C')
    return mesh

In [5]:
def run_1d_tem_inversion(args):
    sig0, dobs, std, eps, rxloc, srcloc, times = args
    mesh = get_1d_cyl_mesh()
    active = mesh.vectorCCz < 0.
    actMap = Maps.InjectActiveCells(mesh, active, np.log(1e-8), nC=mesh.nCz)
    mapping = Maps.ExpMap(mesh) * Maps.SurjectVertical1D(mesh) * actMap
    rx = EM.TDEM.Rx.Point_dbdt(
        rxloc,
        times,
        'z'
    )
    src = EM.TDEM.Src.CircularLoop([rx], orientation='z', loc=srcloc)
    survey = EM.TDEM.Survey([src])
    prb = EM.TDEM.Problem3D_b(mesh, sigmaMap=mapping)

    prb.Solver = Pardiso
    prb.timeSteps = [(1e-05, 15), (5e-5, 10), (2e-4, 10)]
    prb.pair(survey)
    survey.dobs = dobs
    survey.std = std
    survey.eps = eps

    dmisfit = DataMisfit.l2_DataMisfit(survey)
    regMesh = Mesh.TensorMesh([mesh.hz[mapping.maps[-1].indActive]])
    reg = Regularization.Tikhonov(
        regMesh, alpha_s=1./mesh.hx.min()**2, alpha_x=1.
    )
    opt = Optimization.InexactGaussNewton(maxIter=10, LSshorten=0.5)
    invProb = InvProblem.BaseInvProblem(dmisfit, reg, opt, beta=20)
    # Create an inversion object
#     beta = Directives.BetaSchedule(coolingFactor=2, coolingRate=2)
#     betaest = Directives.BetaEstimate_ByEig(beta0_ratio=1e0)
    target=Directives.TargetMisfit()
    inv = Inversion.BaseInversion(invProb, directiveList=[target])
    m0 = np.log(np.ones(actMap.nP)*sig0)
    prb.counter = opt.counter = Utils.Counter()
    mopt = inv.run(m0)
    return mopt, invProb.dpred

## Run the 1D inversions

In [6]:
from multiprocessing import Pool

In [7]:
p = Pool()

SimPEG.InvProblem will set Regularization.mref to m0.
SimPEG.InvProblem will set Regularization.mref to m0.
SimPEG.InvProblem will set Regularization.mref to m0.

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***SimPEG.InvProblem will set Regularization.mref to m0.

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***

model has any nan: 0
model has any nan: 0
============================ Inexact Gauss Newton ============================
============================ Inexact Gauss Newton ============================
  #     beta     phi_d     phi_m

Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-2:
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-8cb311aaf1eb>", line 37, in run_1d_tem_inversion
    mopt = inv.run(m0)
Traceback (most recent call last):
  File "/anaconda

  File "/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py", line 98, in __init__
    self.check_format(full_check=False)
  File "/Users/lindseyjh/git/python_symlinks/SimPEG/Utils/CounterUtils.py", line 88, in wrapper
    out = f(self, *args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py", line 172, in check_format
    self.prune()
  File "/Users/lindseyjh/git/python_symlinks/SimPEG/Survey.py", line 461, in residual
    return Utils.mkvc(self.dpred(m, f=f) - self.dobs)
  File "/Users/lindseyjh/git/python_symlinks/SimPEG/Utils/CounterUtils.py", line 99, in wrapper
    out = f(self, *args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py", line 1075, in prune
    self.data = _prune_array(self.data[:self.nnz])
  File "/Users/lindseyjh/git/python_symlinks/SimPEG/Inversion.py", line 67, in run
    self.m = self.opt.minimize(self.invProb.evalFunction, self.invProb.model)
  File "/anaconda3/lib/python3.6/s

In [8]:
sig0 = 1e-3
std = 0.05
eps = 0.
rxloc = np.array([[0., 0., 30.]])
srcloc = np.array([[0., 0., 30.]])
results = p.map(run_1d_tem_inversion, [(sig0, DOBS[iSrc, 0, :], std, eps, rxloc, srcloc, times) for iSrc in range(len(x))])

SimPEG.InvProblem will set Regularization.mref to m0.

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***
SimPEG.InvProblem will set Regularization.mref to m0.

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***
SimPEG.InvProblem will set Regularization.mref to m0.
SimPEG.InvProblem will set Regularization.mref to m0.

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***
model has any nan: 0
============================ Inexact Gauss Newton ============================
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
model has any nan: 0
============================ Inex

KeyboardInterrupt: 

============================ Inexact Gauss Newton ============================
x0 has any nan: 0
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
-----------------------------------------------------------------------------
model has any nan: 0
============================ Inexact Gauss Newton ============================
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
-----------------------------------------------------------------------------
x0 has any nan: 0
   0  2.00e+01  3.20e+03  0.00e+00  3.20e+03    4.75e+02      0              
   0  2.00e+01  3.23e+03  0.00e+00  3.23e+03    3.20e+02      0              
   0  2.00e+01  2.36e+03  0.00e+00  2.36e+03    7.21e+02      0              
   0  2.00e+01  1.43e+03  0.00e+00  1.43e+03    7.82e+02      0              
   1  2.00e+01  1.56e+02  1.04e-01  1.58e+02    7.50e+01      1              
   1  2.00e+01  8.13e+02  1.11e-01  8.16e+02    8.34e+02      2              
   1 

   2  2.00e+01  2.70e+02  8.26e-02  2.72e+02    9.93e+01      2              
   0  2.00e+01  3.21e+03  0.00e+00  3.21e+03    3.05e+02      0              
   4  2.00e+01  7.09e+01  2.27e-01  7.55e+01    2.40e+02      1   Skip BFGS  
   4  2.00e+01  9.88e+01  1.53e-01  1.02e+02    1.62e+02      2   Skip BFGS  
   5  2.00e+01  6.94e+01  2.23e-01  7.39e+01    3.43e+02      0              
   3  2.00e+01  2.57e+02  1.51e-01  2.60e+02    1.41e+02      3   Skip BFGS  


## Predicted data and resultant model

In [ ]:
DPRED = [results[i][1] for i in range (nSrc)]
MOPT = [results[i][0] for i in range (nSrc)]

In [ ]:
pred = np.vstack(DPRED)
sigma = Utils.mkvc(np.exp(np.vstack(MOPT)))

In [ ]:
dobs = DOBS[:, 0, :].flatten()
dpred = pred.flatten()
W = Utils.sdiag(1./(dobs*std + eps))
misfit = W*(dobs-dpred)
misfit = misfit.dot(misfit)

print('global misfit {:1.0f}, target misfit {:1.0f}'.format(misfit, len(dobs)))
print('chifact: {:1.1f}'.format(misfit/len(dobs)))

In [ ]:
# fig = plt.figure(figsize=(5, 3))
fig = plt.figure(figsize=(9, 4))
for i in range(times.size):
    plt.plot(x, -DOBS[:, 0, i].flatten(), 'k-', lw=1, label='obs' if i == 0 else None)
    plt.plot(x, -pred[:,i].flatten(), 'bx', markeredgewidth=1.5, label='pred' if i == 0 else None)    
plt.yscale('log')   
plt.grid(which="both")
plt.legend(bbox_to_anchor=[1, 1])
plt.xlabel('x')
plt.ylabel('Voltage (V/Am$^2$)')
# plt.ylim(1e-15, 1e-9)

In [ ]:
mesh1d = get_1d_cyl_mesh()
hx = np.ones(nSrc) * 50
mesh2D = Mesh.TensorMesh([hx, mesh1d.hz[mesh1d.vectorCCz<0.]], x0="CN")

In [ ]:
def rect2D(p1, p2):    
    xy = np.c_[np.r_[p1[0], p2[0], p2[0], p1[0], p1[0]], np.r_[p1[1], p1[1], p2[1], p2[1], p1[1]]]
    return xy

p1 = np.r_[-50, -450]
p2 = np.r_[50, -50]
xy_box = rect2D(p1, p2)

In [ ]:
from matplotlib.colors import LogNorm

fig, ax = plt.subplots(2, 1, figsize=(8, 3.5*2))

out = mesh2D.plotImage(sigma, pcolorOpts={'norm':LogNorm()}, clim=[1e-3, 1e-1], ax=ax[0])
cb = plt.colorbar(out[0], fraction=0.025, extend="min", ax=ax[0])
ax[0].set_ylim(-500, 0.)
plt.gca().set_aspect(1)
cb.set_label('Conductivity (S/m)')
ax[0].set_title("(a)")

ax[0].plot(xy_box[:,0], xy_box[:,1], 'w--')

for i in range(times.size):
    ax[1].plot(x, -DOBS[:, 0, i].flatten(), 'k-', lw=1, label='obs' if i == 0 else None)
    ax[1].plot(x, -pred[:,i].flatten(), 'bx', markeredgewidth=1.5, label='pred' if i == 0 else None)    
ax[1].set_yscale('log')   
ax[1].grid(which="both")
ax[1].legend(bbox_to_anchor=[1, 1])
ax[1].set_xlabel('x')
ax[1].set_ylabel('Voltage (V/Am$^2$)')
ax[1].set_title("(b)")

plt.tight_layout()

fig.savefig('../figures/1Dinversion.png', dpi=350)

In [ ]:
np.save('1Dinversion/dpred_1d_stitched', pred)
np.save('1Dinversion/sigma_1d_stitched', sigma)
mesh2D.save(filename="1Dinversion/mesh_1d_stitched.json")